In [1]:
import tkinter as tk
from tkinter import messagebox, ttk
import json
from datetime import date, datetime
from docx import Document
import os

# Date string to date object conversion
def str_to_date(obj):
    if isinstance(obj, str):
        try:
            return datetime.strptime(obj, "%Y-%m-%d").date()
        except ValueError:
            return obj
    return obj

# Medical History class
class MedicalHistory:
    def __init__(self, conditions: list, surgeries: list, allergies: list):
        self.conditions = conditions
        self.surgeries = surgeries
        self.allergies = allergies

    def __str__(self):
        return (f"Conditions: {', '.join(self.conditions)}\n"
                f"Surgeries: {', '.join(self.surgeries)}\n"
                f"Allergies: {', '.join(self.allergies)}")

    def to_dict(self):
        return {
            "conditions": self.conditions,
            "surgeries": self.surgeries,
            "allergies": self.allergies
        }

# Diagnosis class
class Diagnosis:
    def __init__(self, diagnosis: str, icd_code: str):
        self.diagnosis = diagnosis
        self.icd_code = icd_code

    def __str__(self):
        return f"Diagnosis: {self.diagnosis}, ICD Code: {self.icd_code}"

    def to_dict(self):
        return {
            "diagnosis": self.diagnosis,
            "icd_code": self.icd_code
        }

# Treatment Plan class
class TreatmentPlan:
    def __init__(self, exercises: list, therapies: list, modalities: list, interventions: str, home_exercise_program: str):
        self.exercises = exercises
        self.therapies = therapies
        self.modalities = modalities
        self.interventions = interventions
        self.home_exercise_program = home_exercise_program

    def __str__(self):
        return (f"Exercises: {', '.join(self.exercises)}\n"
                f"Therapies: {', '.join(self.therapies)}\n"
                f"Modalities: {', '.join(self.modalities)}\n"
                f"Interventions: {self.interventions}\n"
                f"Home Exercise Program: {self.home_exercise_program}")

    def to_dict(self):
        return {
            "exercises": self.exercises,
            "therapies": self.therapies,
            "modalities": self.modalities,
            "interventions": self.interventions,
            "home_exercise_program": self.home_exercise_program
        }

# Assessment class with treatment_notes
class Assessment:
    def __init__(self, findings: str, assessment_date: date, subjective_symptoms: str, objective_findings: str, history: str, treatment_notes: str = "N/A"):
        self.findings = findings
        self.assessment_date = assessment_date
        self.subjective_symptoms = subjective_symptoms
        self.objective_findings = objective_findings
        self.history = history
        self.treatment_notes = treatment_notes

    def __str__(self):
        return (f"Assessment on {self.assessment_date}: {self.findings}\n"
                f"Subjective: {self.subjective_symptoms}\n"
                f"Objective: {self.objective_findings}\n"
                f"History: {self.history}\n"
                f"Treatment Notes: {self.treatment_notes}")

    def to_dict(self):
        return {
            "findings": self.findings,
            "assessment_date": self.assessment_date,
            "subjective_symptoms": self.subjective_symptoms,
            "objective_findings": self.objective_findings,
            "history": self.history,
            "treatment_notes": self.treatment_notes
        }

# Reassessment class
class Reassessment:
    def __init__(self, date: date, progress_notes: str, objective_findings: str, recommendations: str):
        self.reassessment_date = date
        self.progress_notes = progress_notes
        self.objective_findings = objective_findings
        self.recommendations = recommendations

    def __str__(self):
        return (f"Reassessment on {self.reassessment_date}: {self.progress_notes}\n"
                f"Objective: {self.objective_findings}\n"
                f"Recommendations: {self.recommendations}")

    def to_dict(self):
        return {
            "reassessment_date": self.reassessment_date,
            "progress_notes": self.progress_notes,
            "objective_findings": self.objective_findings,
            "recommendations": self.recommendations
        }

# Patient class
class Patient:
    def __init__(self, name: str, contact: str, dob: date, gender: str, medical_aid: str,
                 medical_aid_number: str, occupation: str, referral_diagnosis: str,
                 physiotherapist_name: str, medical_history: MedicalHistory,
                 diagnosis: Diagnosis, treatment_plan: TreatmentPlan):
        self.name = name
        self.contact = contact
        self.dob = dob
        self.gender = gender
        self.medical_aid = medical_aid
        self.medical_aid_number = medical_aid_number
        self.occupation = occupation
        self.referral_diagnosis = referral_diagnosis
        self.physiotherapist_name = physiotherapist_name
        self.medical_history = medical_history
        self.diagnosis = diagnosis
        self.treatment_plan = treatment_plan
        self.assessments = []
        self.reassessments = []

    def add_assessment(self, assessment: Assessment):
        self.assessments.append(assessment)

    def add_reassessment(self, reassessment: Reassessment):
        self.reassessments.append(reassessment)

    def __str__(self):
        return (f"Patient: {self.name}\n"
                f"Contact: {self.contact}\n"
                f"DOB: {self.dob}\n"
                f"Gender: {self.gender}\n"
                f"Medical Aid: {self.medical_aid}, Number: {self.medical_aid_number}\n"
                f"Occupation: {self.occupation}\n"
                f"Referral Diagnosis: {self.referral_diagnosis}\n"
                f"Physiotherapist: {self.physiotherapist_name}\n"
                f"{self.medical_history}\n"
                f"{self.diagnosis}\n"
                f"{self.treatment_plan}")

    def to_dict(self):
        return {
            "name": self.name,
            "contact": self.contact,
            "dob": self.dob,
            "gender": self.gender,
            "medical_aid": self.medical_aid,
            "medical_aid_number": self.medical_aid_number,
            "occupation": self.occupation,
            "referral_diagnosis": self.referral_diagnosis,
            "physiotherapist_name": self.physiotherapist_name,
            "medical_history": self.medical_history.to_dict(),
            "diagnosis": self.diagnosis.to_dict(),
            "treatment_plan": self.treatment_plan.to_dict(),
            "assessments": [assessment.to_dict() for assessment in self.assessments],
            "reassessments": [reassessment.to_dict() for reassessment in self.reassessments]
        }

    @classmethod
    def from_dict(cls, data):
        medical_history = MedicalHistory(**data["medical_history"])
        diagnosis = Diagnosis(**data["diagnosis"])
        treatment_plan_data = data["treatment_plan"]
        treatment_plan = TreatmentPlan(
            treatment_plan_data.get("exercises", ["none"]),
            treatment_plan_data.get("therapies", ["none"]),
            treatment_plan_data.get("modalities", ["none"]),
            treatment_plan_data.get("interventions", "Initial treatment"),
            treatment_plan_data.get("home_exercise_program", "Home exercises prescribed")
        )
        patient = cls(
            data["name"], data["contact"], data["dob"], data["gender"], data["medical_aid"],
            data["medical_aid_number"], data["occupation"], data["referral_diagnosis"],
            data["physiotherapist_name"], medical_history, diagnosis, treatment_plan
        )
        # Handle assessments with optional treatment_notes
        patient.assessments = []
        for assessment in data.get("assessments", []):
            try:
                patient.assessments.append(
                    Assessment(
                        assessment.get("findings", ""),
                        assessment.get("assessment_date", date.today()),
                        assessment.get("subjective_symptoms", ""),
                        assessment.get("objective_findings", ""),
                        assessment.get("history", "N/A"),
                        assessment.get("treatment_notes", "N/A")
                    )
                )
            except Exception as e:
                print(f"Skipping assessment due to error: {str(e)}")
        # Handle reassessments with potential reassessment_date or date key
        patient.reassessments = []
        for reassessment in data.get("reassessments", []):
            try:
                patient.reassessments.append(
                    Reassessment(
                        reassessment.get("reassessment_date", reassessment.get("date", date.today())),
                        reassessment.get("progress_notes", ""),
                        reassessment.get("objective_findings", ""),
                        reassessment.get("recommendations", "")
                    )
                )
            except Exception as e:
                print(f"Skipping reassessment due to error: {str(e)}")
        return patient

# Main Application class
class PhysioApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Physiotherapy Patient Management System")
        self.root.geometry("800x600")  # Default window size
        self.patients = {}
        self.patient_counter = 1
        self.current_patient = None
        self.frames = {}
        self.patient_var = tk.StringVar()

        # Create main container
        self.container = ttk.Frame(self.root)
        self.container.pack(fill="both", expand=True)

        # Initialize main menu
        self.create_main_menu()

        # Load patients
        self.load_patients()

    def create_main_menu(self):
        self.clear_container()
        frame = ttk.Frame(self.container)
        frame.pack(fill="both", expand=True)

        tk.Label(frame, text="Physiotherapy Patient Management System", font=("Arial", 16)).pack(pady=20)
        tk.Button(frame, text="Patient Management", command=self.create_patient_frame).pack(pady=10)
        tk.Button(frame, text="Assessment & Reassessment", command=self.create_assessment_frame).pack(pady=10)
        tk.Button(frame, text="Generate Report", command=self.create_report_frame).pack(pady=10)
        tk.Button(frame, text="Exit", command=self.root.destroy).pack(pady=10)
        #tk.Button(frame, text="Exit", command=self.root.quit).pack(pady=10)

    def clear_container(self):
        for widget in self.container.winfo_children():
            widget.destroy()
        self.frames.clear()  # Clear stored frames

    def create_patient_frame(self):
        self.clear_container()
        # Create scrollable canvas
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        self.frames["patient"] = scrollable_frame

        # Patient details
        fields = [
            ("Name:", tk.Entry), ("Contact:", tk.Entry), ("DOB (YYYY-MM-DD):", tk.Entry),
            ("Gender:", tk.Entry), ("Medical Aid:", tk.Entry), ("Medical Aid Number:", tk.Entry),
            ("Occupation:", tk.Entry), ("Referral Diagnosis:", tk.Entry), ("Physiotherapist Name:", tk.Entry)
        ]
        self.patient_entries = {}
        for i, (label, widget_type) in enumerate(fields):
            tk.Label(scrollable_frame, text=label).grid(row=i, column=0, padx=5, pady=5, sticky="w")
            entry = widget_type(scrollable_frame)
            entry.grid(row=i, column=1, padx=5, pady=5, sticky="ew")
            self.patient_entries[label.strip(":")] = entry

        # Medical History
        tk.Label(scrollable_frame, text="Medical History").grid(row=len(fields), column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text="Conditions (comma-separated):").grid(row=len(fields)+1, column=0, padx=5, pady=5, sticky="w")
        self.conditions_entry = tk.Entry(scrollable_frame)
        self.conditions_entry.grid(row=len(fields)+1, column=1, padx=5, pady=5, sticky="ew")
        tk.Label(scrollable_frame, text="Surgeries (comma-separated):").grid(row=len(fields)+2, column=0, padx=5, pady=5, sticky="w")
        self.surgeries_entry = tk.Entry(scrollable_frame)
        self.surgeries_entry.grid(row=len(fields)+2, column=1, padx=5, pady=5, sticky="ew")
        tk.Label(scrollable_frame, text="Allergies (comma-separated):").grid(row=len(fields)+3, column=0, padx=5, pady=5, sticky="w")
        self.allergies_entry = tk.Entry(scrollable_frame)
        self.allergies_entry.grid(row=len(fields)+3, column=1, padx=5, pady=5, sticky="ew")

        # Diagnosis
        tk.Label(scrollable_frame, text="Diagnosis").grid(row=len(fields)+4, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text="Diagnosis:").grid(row=len(fields)+5, column=0, padx=5, pady=5, sticky="w")
        self.diagnosis_entry = tk.Entry(scrollable_frame)
        self.diagnosis_entry.grid(row=len(fields)+5, column=1, padx=5, pady=5, sticky="ew")
        tk.Label(scrollable_frame, text="ICD Code:").grid(row=len(fields)+6, column=0, padx=5, pady=5, sticky="w")
        self.icd_code_entry = tk.Entry(scrollable_frame)
        self.icd_code_entry.grid(row=len(fields)+6, column=1, padx=5, pady=5, sticky="ew")

        # Treatment Plan
        tk.Label(scrollable_frame, text="Treatment Plan").grid(row=len(fields)+7, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text="Exercises (comma-separated):").grid(row=len(fields)+8, column=0, padx=5, pady=5, sticky="w")
        self.exercises_entry = tk.Entry(scrollable_frame)
        self.exercises_entry.grid(row=len(fields)+8, column=1, padx=5, pady=5, sticky="ew")
        tk.Label(scrollable_frame, text="Therapies (comma-separated):").grid(row=len(fields)+9, column=0, padx=5, pady=5, sticky="w")
        self.therapies_entry = tk.Entry(scrollable_frame)
        self.therapies_entry.grid(row=len(fields)+9, column=1, padx=5, pady=5, sticky="ew")
        tk.Label(scrollable_frame, text="Modalities (comma-separated):").grid(row=len(fields)+10, column=0, padx=5, pady=5, sticky="w")
        self.modalities_entry = tk.Entry(scrollable_frame)
        self.modalities_entry.grid(row=len(fields)+10, column=1, padx=5, pady=5, sticky="ew")
        tk.Label(scrollable_frame, text="Interventions:").grid(row=len(fields)+11, column=0, padx=5, pady=5, sticky="w")
        self.interventions_entry = tk.Entry(scrollable_frame)
        self.interventions_entry.grid(row=len(fields)+11, column=1, padx=5, pady=5, sticky="ew")
        tk.Label(scrollable_frame, text="Home Exercise Program:").grid(row=len(fields)+12, column=0, padx=5, pady=5, sticky="w")
        self.home_exercise_entry = tk.Entry(scrollable_frame)
        self.home_exercise_entry.grid(row=len(fields)+12, column=1, padx=5, pady=5, sticky="ew")

        tk.Button(scrollable_frame, text="Add Patient", command=self.add_patient).grid(row=len(fields)+13, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text="Back to Menu", command=self.create_main_menu, width=20).grid(row=len(fields)+14, column=0, columnspan=2, pady=20)

        # Configure column weight
        scrollable_frame.columnconfigure(1, weight=1)

    def create_assessment_frame(self):
        self.clear_container()
        # Create scrollable canvas
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        self.frames["assessment"] = scrollable_frame

        # Patient selection
        tk.Label(scrollable_frame, text="Select Patient:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.assess_patient_menu = tk.OptionMenu(scrollable_frame, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.assess_patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        # Assessment fields
        tk.Label(scrollable_frame, text="Assessment Findings:").grid(row=1, column=0, padx=5, pady=5, sticky="w")
        self.assessment_findings_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.assessment_findings_entry.grid(row=1, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(scrollable_frame, text="Subjective Symptoms:").grid(row=2, column=0, padx=5, pady=5, sticky="w")
        self.subjective_symptoms_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.subjective_symptoms_entry.grid(row=2, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(scrollable_frame, text="Objective Findings:").grid(row=3, column=0, padx=5, pady=5, sticky="w")
        self.objective_findings_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.objective_findings_entry.grid(row=3, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(scrollable_frame, text="History:").grid(row=4, column=0, padx=5, pady=5, sticky="w")
        self.history_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.history_entry.grid(row=4, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(scrollable_frame, text="Treatment Notes (Session):").grid(row=5, column=0, padx=5, pady=5, sticky="w")
        self.treatment_notes_entry = tk.Text(scrollable_frame, height=5, width=40)
        self.treatment_notes_entry.grid(row=5, column=1, padx=5, pady=5, sticky="ew")

        # Reassessment fields
        tk.Label(scrollable_frame, text="Reassessment Progress Notes:").grid(row=6, column=0, padx=5, pady=5, sticky="w")
        self.reassessment_progress_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.reassessment_progress_entry.grid(row=6, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(scrollable_frame, text="Reassessment Objective Findings:").grid(row=7, column=0, padx=5, pady=5, sticky="w")
        self.reassessment_objective_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.reassessment_objective_entry.grid(row=7, column=1, padx=5, pady=5, sticky="ew")

        tk.Label(scrollable_frame, text="Recommendations:").grid(row=8, column=0, padx=5, pady=5, sticky="w")
        self.recommendations_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.recommendations_entry.grid(row=8, column=1, padx=5, pady=5, sticky="ew")

        # Buttons
        tk.Button(scrollable_frame, text="Add Assessment", command=self.add_assessment).grid(row=9, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text="Add Reassessment", command=self.add_reassessment).grid(row=10, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text="Back to Menu", command=self.create_main_menu, width=20).grid(row=11, column=0, columnspan=2, pady=20)

        # Configure column weight
        scrollable_frame.columnconfigure(1, weight=1)

    def create_report_frame(self):
        self.clear_container()
        # Create scrollable canvas
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        self.frames["report"] = scrollable_frame

        tk.Label(scrollable_frame, text="Select Patient for Report:").grid(row=0, column=0, padx=5, pady=5, sticky="w")
        self.report_patient_menu = tk.OptionMenu(scrollable_frame, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.report_patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky="ew")

        tk.Button(scrollable_frame, text="Generate Word Report", command=self.generate_word_report).grid(row=1, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text="Back to Menu", command=self.create_main_menu, width=20).grid(row=2, column=0, columnspan=2, pady=20)

        # Configure column weight
        scrollable_frame.columnconfigure(1, weight=1)

    def add_patient(self):
        try:
            name = self.patient_entries["Name"].get()
            contact = self.patient_entries["Contact"].get()
            dob_str = self.patient_entries["DOB (YYYY-MM-DD)"].get()
            gender = self.patient_entries["Gender"].get()
            medical_aid = self.patient_entries["Medical Aid"].get()
            medical_aid_number = self.patient_entries["Medical Aid Number"].get()
            occupation = self.patient_entries["Occupation"]. Каждое
            referral_diagnosis = self.patient_entries["Referral Diagnosis"].get()
            physiotherapist_name = self.patient_entries["Physiotherapist Name"].get()

            try:
                dob = datetime.strptime(dob_str, "%Y-%m-%d").date()
            except ValueError:
                messagebox.showerror("Error", "Invalid DOB format! Use YYYY-MM-DD.")
                return

            conditions = [c.strip() for c in self.conditions_entry.get().split(",") if c.strip()] or ["none"]
            surgeries = [s.strip() for s in self.surgeries_entry.get().split(",") if s.strip()] or ["none"]
            allergies = [a.strip() for a in self.allergies_entry.get().split(",") if a.strip()] or ["none"]
            diagnosis = self.diagnosis_entry.get() or "N/A"
            icd_code = self.icd_code_entry.get() or "N/A"
            exercises = [e.strip() for e in self.exercises_entry.get().split(",") if e.strip()] or ["none"]
            therapies = [t.strip() for t in self.therapies_entry.get().split(",") if t.strip()] or ["none"]
            modalities = [m.strip() for m in self.modalities_entry.get().split(",") if m.strip()] or ["none"]
            interventions = self.interventions_entry.get() or "Initial treatment"
            home_exercise = self.home_exercise_entry.get() or "Home exercises prescribed"

            if not name or not referral_diagnosis or not physiotherapist_name:
                messagebox.showerror("Error", "Name, referral diagnosis, and physiotherapist name are required!")
                return

            medical_history = MedicalHistory(conditions, surgeries, allergies)
            diagnosis_obj = Diagnosis(diagnosis, icd_code)
            treatment_plan = TreatmentPlan(exercises, therapies, modalities, interventions, home_exercise)

            patient = Patient(
                name, contact, dob, gender, medical_aid, medical_aid_number, occupation,
                referral_diagnosis, physiotherapist_name, medical_history, diagnosis_obj, treatment_plan
            )

            patient_id = f"patient_{self.patient_counter:03d}"
            self.patients[patient_id] = patient
            self.patient_counter += 1
            self.current_patient = patient
            self.save_patients()
            messagebox.showinfo("Success", f"Patient {name} added!")
            self.clear_patient_entries()

        except Exception as e:
            messagebox.showerror("Error", f"Failed to add patient: {str(e)}")

    def clear_patient_entries(self):
        for entry in self.patient_entries.values():
            entry.delete(0, tk.END)
        self.conditions_entry.delete(0, tk.END)
        self.surgeries_entry.delete(0, tk.END)
        self.allergies_entry.delete(0, tk.END)
        self.diagnosis_entry.delete(0, tk.END)
        self.icd_code_entry.delete(0, tk.END)
        self.exercises_entry.delete(0, tk.END)
        self.therapies_entry.delete(0, tk.END)
        self.modalities_entry.delete(0, tk.END)
        self.interventions_entry.delete(0, tk.END)
        self.home_exercise_entry.delete(0, tk.END)

    def add_assessment(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return

        findings = self.assessment_findings_entry.get("1.0", tk.END).strip()
        subjective_symptoms = self.subjective_symptoms_entry.get("1.0", tk.END).strip()
        objective_findings = self.objective_findings_entry.get("1.0", tk.END).strip()
        history = self.history_entry.get("1.0", tk.END).strip()
        treatment_notes = self.treatment_notes_entry.get("1.0", tk.END).strip()

        if not all([findings, subjective_symptoms, objective_findings]):
            messagebox.showerror("Error", "Findings, Subjective Symptoms, and Objective Findings are required!")
            return

        assessment = Assessment(findings, date.today(), subjective_symptoms, objective_findings, history or "N/A", treatment_notes or "N/A")
        self.current_patient.add_assessment(assessment)
        self.save_patients()
        messagebox.showinfo("Success", "Assessment added!")

        self.assessment_findings_entry.delete("1.0", tk.END)
        self.subjective_symptoms_entry.delete("1.0", tk.END)
        self.objective_findings_entry.delete("1.0", tk.END)
        self.history_entry.delete("1.0", tk.END)
        self.treatment_notes_entry.delete("1.0", tk.END)

    def add_reassessment(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return

        progress_notes = self.reassessment_progress_entry.get("1.0", tk.END).strip()
        objective_findings = self.reassessment_objective_entry.get("1.0", tk.END).strip()
        recommendations = self.recommendations_entry.get("1.0", tk.END).strip()

        if not all([progress_notes, objective_findings]):
            messagebox.showerror("Error", "Progress Notes and Objective Findings are required!")
            return

        reassessment = Reassessment(date.today(), progress_notes, objective_findings, recommendations or "N/A")
        self.current_patient.add_reassessment(reassessment)
        self.save_patients()
        messagebox.showinfo("Success", "Reassessment added!")

        self.reassessment_progress_entry.delete("1.0", tk.END)
        self.reassessment_objective_entry.delete("1.0", tk.END)
        self.recommendations_entry.delete("1.0", tk.END)

    def generate_word_report(self):
        if not self.current_patient:
            messagebox.showerror("Error", "No patient selected!")
            return

        document = Document()
        document.add_heading(f"Physiotherapy Management Report for {self.current_patient.name}", level=0)

        document.add_paragraph(f"Name: {self.current_patient.name}")
        document.add_paragraph(f"Date of Birth: {self.current_patient.dob}")
        document.add_paragraph(f"Medical Aid: {self.current_patient.medical_aid}")
        document.add_paragraph(f"Medical Aid Number: {self.current_patient.medical_aid_number}")
        document.add_paragraph(f"Occupation: {self.current_patient.occupation}")
        document.add_paragraph(f"Physiotherapist: {self.current_patient.physiotherapist_name}")
        document.add_paragraph(f"Referral Diagnosis: {self.current_patient.referral_diagnosis}")

        document.add_paragraph("Dear Dr,")
        document.add_paragraph(f"Thank you for the referral of the above-mentioned patient.")

        age = (datetime.now().date() - self.current_patient.dob).days // 365
        age_str = f"{age}-year-old" if age >= 1 else f"{(datetime.now().date() - self.current_patient.dob).days // 30}-month-old"
        document.add_paragraph(
            f"The patient is a {age_str} {self.current_patient.gender.lower()} who presented "
            f"with {self.current_patient.referral_diagnosis.lower()}."
        )

        if self.current_patient.assessments:
            latest_assessment = self.current_patient.assessments[-1]
            document.add_heading("Initial Assessment", level=1)
            document.add_paragraph(
                f"On assessment, the patient reported {latest_assessment.subjective_symptoms.lower()}. "
                f"Objective assessment revealed {latest_assessment.objective_findings.lower()}. "
                f"Relevant history includes {latest_assessment.history.lower()}."
            )
        else:
            document.add_paragraph("No assessment recorded.")

        document.add_heading("Treatment", level=1)
        treatment_details = f"Physiotherapy management included {self.current_patient.treatment_plan.interventions.lower()}."
        if self.current_patient.assessments and self.current_patient.assessments[-1].treatment_notes != "N/A":
            treatment_details += f" During the session, the following treatments were applied: {self.current_patient.assessments[-1].treatment_notes.lower()}."
        treatment_details += f" A home exercise program was provided, including {self.current_patient.treatment_plan.home_exercise_program.lower()}."
        document.add_paragraph(treatment_details)

        if self.current_patient.reassessments:
            latest_reassessment = self.current_patient.reassessments[-1]
            document.add_heading("Reassessment", level=1)
            document.add_paragraph(
                f"On reassessment, the patient reported {latest_reassessment.progress_notes.lower()}. "
                f"Objective findings included {latest_reassessment.objective_findings.lower()}. "
                f"{latest_reassessment.recommendations}."
            )
        else:
            document.add_paragraph("No reassessment recorded.")

        document.add_paragraph("Kind regards,")
        document.add_paragraph(self.current_patient.physiotherapist_name)
        document.add_paragraph("Physiotherapist")

        try:
            filename = f"{self.current_patient.name.replace(' ', '_')}_report.docx"
            document.save(filename)
            messagebox.showinfo("Success", f"Report saved as '{filename}'")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save report: {str(e)}")

    def get_patient_names(self):
        return [patient.name for patient in self.patients.values()] or ["No patients"]

    def select_patient(self, *args):
        patient_name = self.patient_var.get()
        self.current_patient = next((p for p in self.patients.values() if p.name == patient_name), None)

    def save_patients(self):
        try:
            patients_data = {pid: patient.to_dict() for pid, patient in self.patients.items()}
            with open("patients.json", "w") as f:
                json.dump(patients_data, f, default=str)
        except Exception as e:
            messagebox.showerror("Error", f"Failed to save patients: {str(e)}")

    def load_patients(self):
        try:
            with open("patients.json", "r") as f:
                patients_data = json.load(f, object_hook=str_to_date)
            self.patients = {}
            for pid, data in patients_data.items():
                try:
                    self.patients[pid] = Patient.from_dict(data)
                except Exception as e:
                    print(f"Error loading patient {pid}: {str(e)}")
                    messagebox.showwarning("Warning", f"Failed to load patient {pid}: {str(e)}")
            self.patient_counter = len(self.patients) + 1
            if self.patients:
                self.current_patient = list(self.patients.values())[-1]
            self.update_patient_menu()
        except FileNotFoundError:
            self.patients = {}
            self.update_patient_menu()
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load patients: {str(e)}")

    def update_patient_menu(self):
        if "assessment" in self.frames:
            menu = self.assess_patient_menu["menu"]
            menu.delete(0, "end")
            for name in self.get_patient_names():
                menu.add_command(label=name, command=lambda n=name: self.patient_var.set(n))
        if "report" in self.frames:
            menu = self.report_patient_menu["menu"]
            menu.delete(0, "end")
            for name in self.get_patient_names():
                menu.add_command(label=name, command=lambda n=name: self.patient_var.set(n))
        if self.get_patient_names() != ["No patients"]:
            self.patient_var.set(self.get_patient_names()[-1])
        else:
            self.patient_var.set("No patients")

if __name__ == "__main__":
    root = tk.Tk()
    app = PhysioApp(root)
    root.mainloop()

In [1]:
#PhysioApp4.2
import tkinter as tk
from tkinter import messagebox, ttk
import json
from datetime import date, datetime
from docx import Document
import os
import logging

# Configure logging for debugging
logging.basicConfig(filename='physio_app.log', level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')

# Date string to date object conversion
def str_to_date(obj):
    if isinstance(obj, str):
        try:
            return datetime.strptime(obj, '%Y-%m-%d').date()
        except ValueError:
            return obj
    return obj

# Medical History class
class MedicalHistory:
    def __init__(self, conditions: list, surgeries: list, allergies: list):
        self.conditions = conditions
        self.surgeries = surgeries
        self.allergies = allergies

    def __str__(self):
        return (f'Conditions: {", ".join(self.conditions)}\n'
                f'Surgeries: {", ".join(self.surgeries)}\n'
                f'Allergies: {", ".join(self.allergies)}')

    def to_dict(self):
        return {
            'conditions': self.conditions,
            'surgeries': self.surgeries,
            'allergies': self.allergies
        }

# Diagnosis class
class Diagnosis:
    def __init__(self, diagnosis: str, icd_code: str):
        self.diagnosis = diagnosis
        self.icd_code = icd_code

    def __str__(self):
        return f'Diagnosis: {self.diagnosis}, ICD Code: {self.icd_code}'

    def to_dict(self):
        return {
            'diagnosis': self.diagnosis,
            'icd_code': self.icd_code
        }

# Treatment Plan class
class TreatmentPlan:
    def __init__(self, exercises: list, therapies: list, modalities: list, interventions: str, home_exercise_program: str):
        self.exercises = exercises
        self.therapies = therapies
        self.modalities = modalities
        self.interventions = interventions
        self.home_exercise_program = home_exercise_program

    def __str__(self):
        return (f'Exercises: {", ".join(self.exercises)}\n'
                f'Therapies: {", ".join(self.therapies)}\n'
                f'Modalities: {", ".join(self.modalities)}\n'
                f'Interventions: {self.interventions}\n'
                f'Home Exercise Program: {self.home_exercise_program}')

    def to_dict(self):
        return {
            'exercises': self.exercises,
            'therapies': self.therapies,
            'modalities': self.modalities,
            'interventions': self.interventions,
            'home_exercise_program': self.home_exercise_program
        }

# Assessment class with treatment_notes
class Assessment:
    def __init__(self, findings: str, assessment_date: date, subjective_symptoms: str, objective_findings: str, history: str, treatment_notes: str = 'N/A'):
        self.findings = findings
        self.assessment_date = assessment_date
        self.subjective_symptoms = subjective_symptoms
        self.objective_findings = objective_findings
        self.history = history
        self.treatment_notes = treatment_notes

    def __str__(self):
        return (f'Assessment on {self.assessment_date}: {self.findings}\n'
                f'Subjective: {self.subjective_symptoms}\n'
                f'Objective: {self.objective_findings}\n'
                f'History: {self.history}\n'
                f'Treatment Notes: {self.treatment_notes}')

    def to_dict(self):
        return {
            'findings': self.findings,
            'assessment_date': self.assessment_date,
            'subjective_symptoms': self.subjective_symptoms,
            'objective_findings': self.objective_findings,
            'history': self.history,
            'treatment_notes': self.treatment_notes
        }

# Reassessment class
class Reassessment:
    def __init__(self, date: date, progress_notes: str, objective_findings: str, recommendations: str):
        self.reassessment_date = date
        self.progress_notes = progress_notes
        self.objective_findings = objective_findings
        self.recommendations = recommendations

    def __str__(self):
        return (f'Reassessment on {self.reassessment_date}: {self.progress_notes}\n'
                f'Objective: {self.objective_findings}\n'
                f'Recommendations: {self.recommendations}')

    def to_dict(self):
        return {
            'reassessment_date': self.reassessment_date,
            'progress_notes': self.progress_notes,
            'objective_findings': self.objective_findings,
            'recommendations': self.recommendations
        }

# Patient class
class Patient:
    def __init__(self, name: str, contact: str, dob: date, gender: str, medical_aid: str,
                 medical_aid_number: str, occupation: str, referral_diagnosis: str,
                 physiotherapist_name: str, medical_history: MedicalHistory,
                 diagnosis: Diagnosis, treatment_plan: TreatmentPlan):
        self.name = name
        self.contact = contact
        self.dob = dob
        self.gender = gender
        self.medical_aid = medical_aid
        self.medical_aid_number = medical_aid_number
        self.occupation = occupation
        self.referral_diagnosis = referral_diagnosis
        self.physiotherapist_name = physiotherapist_name
        self.medical_history = medical_history
        self.diagnosis = diagnosis
        self.treatment_plan = treatment_plan
        self.assessments = []
        self.reassessments = []

    def add_assessment(self, assessment: Assessment):
        self.assessments.append(assessment)

    def add_reassessment(self, reassessment: Reassessment):
        self.reassessments.append(reassessment)

    def __str__(self):
        return (f'Patient: {self.name}\n'
                f'Contact: {self.contact}\n'
                f'DOB: {self.dob}\n'
                f'Gender: {self.gender}\n'
                f'Medical Aid: {self.medical_aid}, Number: {self.medical_aid_number}\n'
                f'Occupation: {self.occupation}\n'
                f'Referral Diagnosis: {self.referral_diagnosis}\n'
                f'Physiotherapist: {self.physiotherapist_name}\n'
                f'{self.medical_history}\n'
                f'{self.diagnosis}\n'
                f'{self.treatment_plan}')

    def to_dict(self):
        return {
            'name': self.name,
            'contact': self.contact,
            'dob': self.dob,
            'gender': self.gender,
            'medical_aid': self.medical_aid,
            'medical_aid_number': self.medical_aid_number,
            'occupation': self.occupation,
            'referral_diagnosis': self.referral_diagnosis,
            'physiotherapist_name': self.physiotherapist_name,
            'medical_history': self.medical_history.to_dict(),
            'diagnosis': self.diagnosis.to_dict(),
            'treatment_plan': self.treatment_plan.to_dict(),
            'assessments': [assessment.to_dict() for assessment in self.assessments],
            'reassessments': [reassessment.to_dict() for reassessment in self.reassessments]
        }

    @classmethod
    def from_dict(cls, data):
        medical_history = MedicalHistory(**data['medical_history'])
        diagnosis = Diagnosis(**data['diagnosis'])
        treatment_plan_data = data['treatment_plan']
        treatment_plan = TreatmentPlan(
            treatment_plan_data.get('exercises', ['none']),
            treatment_plan_data.get('therapies', ['none']),
            treatment_plan_data.get('modalities', ['none']),
            treatment_plan_data.get('interventions', 'Initial treatment'),
            treatment_plan_data.get('home_exercise_program', 'Home exercises prescribed')
        )
        patient = cls(
            data['name'], data['contact'], data['dob'], data['gender'], data['medical_aid'],
            data['medical_aid_number'], data['occupation'], data['referral_diagnosis'],
            data['physiotherapist_name'], medical_history, diagnosis, treatment_plan
        )
        patient.assessments = []
        for assessment in data.get('assessments', []):
            try:
                patient.assessments.append(
                    Assessment(
                        assessment.get('findings', ''),
                        assessment.get('assessment_date', date.today()),
                        assessment.get('subjective_symptoms', ''),
                        assessment.get('objective_findings', ''),
                        assessment.get('history', 'N/A'),
                        assessment.get('treatment_notes', 'N/A')
                    )
                )
            except Exception as e:
                logging.error(f'Skipping assessment due to error: {str(e)}')
        patient.reassessments = []
        for reassessment in data.get('reassessments', []):
            try:
                patient.reassessments.append(
                    Reassessment(
                        reassessment.get('reassessment_date', reassessment.get('date', date.today())),
                        reassessment.get('progress_notes', ''),
                        reassessment.get('objective_findings', ''),
                        reassessment.get('recommendations', '')
                    )
                )
            except Exception as e:
                logging.error(f'Skipping reassessment due to error: {str(e)}')
        return patient

# Main Application class
class PhysioApp:
    def __init__(self, root):
        self.root = root
        self.root.title('Physiotherapy Patient Management System')
        self.root.geometry('800x600')
        self.patients = {}
        self.patient_counter = 1
        self.current_patient = None
        self.frames = {}
        self.patient_var = tk.StringVar()
        self.edit_patient_var = tk.StringVar()

        self.container = ttk.Frame(self.root)
        self.container.pack(fill='both', expand=True)

        self.create_main_menu()
        self.load_patients()

    def bind_scroll_events(self, canvas):
        def on_mouse_wheel(event):
            canvas.yview_scroll(int(-1 * (event.delta / 120)), 'units')
        
        def on_key_scroll(event):
            if event.keysym == 'Up':
                canvas.yview_scroll(-1, 'units')
            elif event.keysym == 'Down':
                canvas.yview_scroll(1, 'units')
            elif event.keysym == 'Prior':
                canvas.yview_scroll(-10, 'units')
            elif event.keysym == 'Next':
                canvas.yview_scroll(10, 'units')
        
        canvas.bind_all('<MouseWheel>', on_mouse_wheel)
        canvas.bind_all('<Button-4>', lambda e: canvas.yview_scroll(-1, 'units'))
        canvas.bind_all('<Button-5>', lambda e: canvas.yview_scroll(1, 'units'))
        canvas.bind_all('<Up>', on_key_scroll)
        canvas.bind_all('<Down>', on_key_scroll)
        canvas.bind_all('<Prior>', on_key_scroll)
        canvas.bind_all('<Next>', on_key_scroll)

    def create_main_menu(self):
        self.clear_container()
        frame = ttk.Frame(self.container)
        frame.pack(fill='both', expand=True)

        tk.Label(frame, text='Physiotherapy Patient Management System', font=('Arial', 16)).pack(pady=20)
        tk.Button(frame, text='Patient Management', command=self.create_patient_frame).pack(pady=10)
        tk.Button(frame, text='Assessment & Reassessment', command=self.create_assessment_frame).pack(pady=10)
        tk.Button(frame, text='Generate Report', command=self.create_report_frame).pack(pady=10)
        tk.Button(frame, text='Exit', command=self.root.destroy).pack(pady=10)

    def clear_container(self):
        for widget in self.container.winfo_children():
            widget.destroy()
        self.frames.clear()

    def create_patient_frame(self):
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient='vertical', command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        scrollable_frame.bind(
            '<Configure>',
            lambda e: canvas.configure(scrollregion=canvas.bbox('all'))
        )
        canvas.create_window((0, 0), window=scrollable_frame, anchor='nw')
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        self.frames['patient'] = scrollable_frame
        self.bind_scroll_events(canvas)

        tk.Label(scrollable_frame, text='Select Patient to Edit:').grid(row=0, column=0, padx=5, pady=5, sticky='w')
        self.edit_patient_menu = tk.OptionMenu(scrollable_frame, self.edit_patient_var, *self.get_patient_names(), command=self.load_patient_details)
        self.edit_patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky='ew')

        fields = [
            ('Name:', tk.Entry), ('Contact:', tk.Entry), ('DOB (YYYY-MM-DD):', tk.Entry),
            ('Gender:', tk.Entry), ('Medical Aid:', tk.Entry), ('Medical Aid Number:', tk.Entry),
            ('Occupation:', tk.Entry), ('Referral Diagnosis:', tk.Entry), ('Physiotherapist Name:', tk.Entry)
        ]
        self.patient_entries = {}
        for i, (label, widget_type) in enumerate(fields):
            tk.Label(scrollable_frame, text=label).grid(row=i+1, column=0, padx=5, pady=5, sticky='w')
            entry = widget_type(scrollable_frame)
            entry.grid(row=i+1, column=1, padx=5, pady=5, sticky='ew')
            self.patient_entries[label.strip(':')] = entry

        tk.Label(scrollable_frame, text='Medical History').grid(row=len(fields)+1, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text='Conditions (comma-separated):').grid(row=len(fields)+2, column=0, padx=5, pady=5, sticky='w')
        self.conditions_entry = tk.Entry(scrollable_frame)
        self.conditions_entry.grid(row=len(fields)+2, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Surgeries (comma-separated):').grid(row=len(fields)+3, column=0, padx=5, pady=5, sticky='w')
        self.surgeries_entry = tk.Entry(scrollable_frame)
        self.surgeries_entry.grid(row=len(fields)+3, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Allergies (comma-separated):').grid(row=len(fields)+4, column=0, padx=5, pady=5, sticky='w')
        self.allergies_entry = tk.Entry(scrollable_frame)
        self.allergies_entry.grid(row=len(fields)+4, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Diagnosis').grid(row=len(fields)+5, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text='Diagnosis:').grid(row=len(fields)+6, column=0, padx=5, pady=5, sticky='w')
        self.diagnosis_entry = tk.Entry(scrollable_frame)
        self.diagnosis_entry.grid(row=len(fields)+6, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='ICD Code:').grid(row=len(fields)+7, column=0, padx=5, pady=5, sticky='w')
        self.icd_code_entry = tk.Entry(scrollable_frame)
        self.icd_code_entry.grid(row=len(fields)+7, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Treatment Plan').grid(row=len(fields)+8, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text='Exercises (comma-separated):').grid(row=len(fields)+9, column=0, padx=5, pady=5, sticky='w')
        self.exercises_entry = tk.Entry(scrollable_frame)
        self.exercises_entry.grid(row=len(fields)+9, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Therapies (comma-separated):').grid(row=len(fields)+10, column=0, padx=5, pady=5, sticky='w')
        self.therapies_entry = tk.Entry(scrollable_frame)
        self.therapies_entry.grid(row=len(fields)+10, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Modalities (comma-separated):').grid(row=len(fields)+11, column=0, padx=5, pady=5, sticky='w')
        self.modalities_entry = tk.Entry(scrollable_frame)
        self.modalities_entry.grid(row=len(fields)+11, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Interventions:').grid(row=len(fields)+12, column=0, padx=5, pady=5, sticky='w')
        self.interventions_entry = tk.Entry(scrollable_frame)
        self.interventions_entry.grid(row=len(fields)+12, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Home Exercise Program:').grid(row=len(fields)+13, column=0, padx=5, pady=5, sticky='w')
        self.home_exercise_entry = tk.Entry(scrollable_frame)
        self.home_exercise_entry.grid(row=len(fields)+13, column=1, padx=5, pady=5, sticky='ew')

        tk.Button(scrollable_frame, text='Add Patient', command=self.add_patient).grid(row=len(fields)+14, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Edit Patient', command=self.edit_patient).grid(row=len(fields)+15, column=0, columnspan=2, pady=10)
        tk.Button(scroll

SyntaxError: incomplete input (41483898.py, line 361)

In [5]:
python -m tabnanny Physio_Narrative_Report4.2.py

SyntaxError: invalid syntax (2823268118.py, line 1)

In [1]:
#PhysioApp4.2
import tkinter as tk
from tkinter import messagebox, ttk
import json
from datetime import date, datetime
from docx import Document
import os
import logging

# Configure logging for debugging
logging.basicConfig(
    filename='physio_app.log',
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Date string to date object conversion
def str_to_date(obj):
    if isinstance(obj, str):
        try:
            return datetime.strptime(obj, '%Y-%m-%d').date()
        except ValueError:
            return obj
    return obj

# Medical History class
class MedicalHistory:
    def __init__(self, conditions: list, surgeries: list, allergies: list):
        self.conditions = conditions
        self.surgeries = surgeries
        self.allergies = allergies

    def __str__(self):
        return (f'Conditions: {", ".join(self.conditions)}\n'
                f'Surgeries: {", ".join(self.surgeries)}\n'
                f'Allergies: {", ".join(self.allergies)}')

    def to_dict(self):
        return {
            'conditions': self.conditions,
            'surgeries': self.surgeries,
            'allergies': self.allergies
        }

# Diagnosis class
class Diagnosis:
    def __init__(self, diagnosis: str, icd_code: str):
        self.diagnosis = diagnosis
        self.icd_code = icd_code

    def __str__(self):
        return f'Diagnosis: {self.diagnosis}, ICD Code: {self.icd_code}'

    def to_dict(self):
        return {
            'diagnosis': self.diagnosis,
            'icd_code': self.icd_code
        }

# Treatment Plan class
class TreatmentPlan:
    def __init__(self, exercises: list, therapies: list, modalities: list,
                 interventions: str, home_exercise_program: str):
        self.exercises = exercises
        self.therapies = therapies
        self.modalities = modalities
        self.interventions = interventions
        self.home_exercise_program = home_exercise_program

    def __str__(self):
        return (f'Exercises: {", ".join(self.exercises)}\n'
                f'Therapies: {", ".join(self.therapies)}\n'
                f'Modalities: {", ".join(self.modalities)}\n'
                f'Interventions: {self.interventions}\n'
                f'Home Exercise Program: {self.home_exercise_program}')

    def to_dict(self):
        return {
            'exercises': self.exercises,
            'therapies': self.therapies,
            'modalities': self.modalities,
            'interventions': self.interventions,
            'home_exercise_program': self.home_exercise_program
        }

# Assessment class with treatment_notes
class Assessment:
    def __init__(self, findings: str, assessment_date: date,
                 subjective_symptoms: str, objective_findings: str,
                 history: str, treatment_notes: str = 'N/A'):
        self.findings = findings
        self.assessment_date = assessment_date
        self.subjective_symptoms = subjective_symptoms
        self.objective_findings = objective_findings
        self.history = history
        self.treatment_notes = treatment_notes

    def __str__(self):
        return (f'Assessment on {self.assessment_date}: {self.findings}\n'
                f'Subjective: {self.subjective_symptoms}\n'
                f'Objective: {self.objective_findings}\n'
                f'History: {self.history}\n'
                f'Treatment Notes: {self.treatment_notes}')

    def to_dict(self):
        return {
            'findings': self.findings,
            'assessment_date': self.assessment_date,
            'subjective_symptoms': self.subjective_symptoms,
            'objective_findings': self.objective_findings,
            'history': self.history,
            'treatment_notes': self.treatment_notes
        }

# Reassessment class
class Reassessment:
    def __init__(self, reassessment_date: date, progress_notes: str,
                 objective_findings: str, recommendations: str):
        self.reassessment_date = reassessment_date
        self.progress_notes = progress_notes
        self.objective_findings = objective_findings
        self.recommendations = recommendations

    def __str__(self):
        return (f'Reassessment on {self.reassessment_date}: {self.progress_notes}\n'
                f'Objective: {self.objective_findings}\n'
                f'Recommendations: {self.recommendations}')

    def to_dict(self):
        return {
            'reassessment_date': self.reassessment_date,
            'progress_notes': self.progress_notes,
            'objective_findings': self.objective_findings,
            'recommendations': self.recommendations
        }

# Patient class
class Patient:
    def __init__(self, name: str, contact: str, dob: date, gender: str,
                 medical_aid: str, medical_aid_number: str, occupation: str,
                 referral_diagnosis: str, physiotherapist_name: str,
                 medical_history: MedicalHistory, diagnosis: Diagnosis,
                 treatment_plan: TreatmentPlan):
        self.name = name
        self.contact = contact
        self.dob = dob
        self.gender = gender
        self.medical_aid = medical_aid
        self.medical_aid_number = medical_aid_number
        self.occupation = occupation
        self.referral_diagnosis = referral_diagnosis
        self.physiotherapist_name = physiotherapist_name
        self.medical_history = medical_history
        self.diagnosis = diagnosis
        self.treatment_plan = treatment_plan
        self.assessments = []
        self.reassessments = []

    def add_assessment(self, assessment: Assessment):
        self.assessments.append(assessment)

    def add_reassessment(self, reassessment: Reassessment):
        self.reassessments.append(reassessment)

    def __str__(self):
        return (f'Patient: {self.name}\n'
                f'Contact: {self.contact}\n'
                f'DOB: {self.dob}\n'
                f'Gender: {self.gender}\n'
                f'Medical Aid: {self.medical_aid}, Number: {self.medical_aid_number}\n'
                f'Occupation: {self.occupation}\n'
                f'Referral Diagnosis: {self.referral_diagnosis}\n'
                f'Physiotherapist: {self.physiotherapist_name}\n'
                f'{self.medical_history}\n'
                f'{self.diagnosis}\n'
                f'{self.treatment_plan}')

    def to_dict(self):
        return {
            'name': self.name,
            'contact': self.contact,
            'dob': self.dob,
            'gender': self.gender,
            'medical_aid': self.medical_aid,
            'medical_aid_number': self.medical_aid_number,
            'occupation': self.occupation,
            'referral_diagnosis': self.referral_diagnosis,
            'physiotherapist_name': self.physiotherapist_name,
            'medical_history': self.medical_history.to_dict(),
            'diagnosis': self.diagnosis.to_dict(),
            'treatment_plan': self.treatment_plan.to_dict(),
            'assessments': [a.to_dict() for a in self.assessments],
            'reassessments': [r.to_dict() for r in self.reassessments]
        }

    @classmethod
    def from_dict(cls, data):
        mh = MedicalHistory(**data.get('medical_history', {}))
        diag = Diagnosis(**data.get('diagnosis', {}))
        tp_data = data.get('treatment_plan', {})
        tp = TreatmentPlan(
            tp_data.get('exercises', []),
            tp_data.get('therapies', []),
            tp_data.get('modalities', []),
            tp_data.get('interventions', ''),
            tp_data.get('home_exercise_program', '')
        )
        patient = cls(
            data.get('name',''), data.get('contact',''), data.get('dob', date.today()),
            data.get('gender',''), data.get('medical_aid',''), data.get('medical_aid_number',''),
            data.get('occupation',''), data.get('referral_diagnosis',''),
            data.get('physiotherapist_name',''), mh, diag, tp
        )
        return patient

# Main Application class
class PhysioApp:
    def __init__(self, root):
        self.root = root
        self.root.title('Physiotherapy Patient Management System')
        self.root.geometry('800x600')
        self.patients = {}
        self.patient_counter = 1
        self.current_patient = None
        self.frames = {}
        self.patient_var = tk.StringVar()
        self.edit_patient_var = tk.StringVar()

        self.container = ttk.Frame(self.root)
        self.container.pack(fill='both', expand=True)

        self.create_main_menu()
        self.load_patients()  # ensure this method is defined below

    def bind_scroll_events(self, canvas):
        def on_mouse_wheel(event):
            canvas.yview_scroll(int(-1 * (event.delta / 120)), 'units')
        def on_key_scroll(event):
            if event.keysym == 'Up':
                canvas.yview_scroll(-1, 'units')
            elif event.keysym == 'Down':
                canvas.yview_scroll(1, 'units')
            elif event.keysym == 'Prior':
                canvas.yview_scroll(-10, 'units')
            elif event.keysym == 'Next':
                canvas.yview_scroll(10, 'units')
        canvas.bind_all('<MouseWheel>', on_mouse_wheel)
        canvas.bind_all('<Button-4>', lambda e: canvas.yview_scroll(-1, 'units'))
        canvas.bind_all('<Button-5>', lambda e: canvas.yview_scroll(1, 'units'))
        canvas.bind_all('<Up>', on_key_scroll)
        canvas.bind_all('<Down>', on_key_scroll)
        canvas.bind_all('<Prior>', on_key_scroll)
        canvas.bind_all('<Next>', on_key_scroll)

    def create_main_menu(self):
        self.clear_container()
        frame = ttk.Frame(self.container)
        frame.pack(fill='both', expand=True)

        tk.Label(frame, text='Physiotherapy Patient Management System', font=('Arial', 16)).pack(pady=20)
        tk.Button(frame, text='Patient Management', command=self.create_patient_frame).pack(pady=10)
        tk.Button(frame, text='Assessment & Reassessment', command=self.create_assessment_frame).pack(pady=10)
        tk.Button(frame, text='Generate Report', command=self.create_report_frame).pack(pady=10)
        tk.Button(frame, text='View Patient History', command=self.view_patient_history).pack(pady=10)
        tk.Button(frame, text='Exit', command=self.root.destroy).pack(pady=10)

    def clear_container(self):
        for widget in self.container.winfo_children():
            widget.destroy()
        self.frames.clear()

    # JSON data loaders
    def load_patients_data(self):
        try:
            with open('patients.json', 'r', encoding='utf-8') as f:
                return json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            messagebox.showerror("Error", "Unable to load patients data.")
            return []

    def load_notes_data(self):
        try:
            with open('notes.json', 'r', encoding='utf-8') as f:
                return json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            messagebox.showerror("Error", "Unable to load notes data.")
            return []

    # View patient history popup
    def view_patient_history(self):
        patients = self.load_patients_data()
        notes = self.load_notes_data()

        history_win = tk.Toplevel(self.root)
        history_win.title("View Patient History")
        history_win.geometry("600x600")

        tk.Label(history_win, text="Enter Patient Name:").pack(pady=5)
        entry = tk.Entry(history_win, width=40)
        entry.pack(pady=5)

        frame = tk.Frame(history_win)
        frame.pack(fill='both', expand=True, pady=10)

        def search():
            for w in frame.winfo_children():
                w.destroy()
            query = entry.get().lower().strip()
            matches = [p for p in patients if query in (p.get('name','') + ' ' + p.get('surname','')).lower()]
            if not matches:
                tk.Label(frame, text="No patients found.", fg="red").pack()
                return
            def show_history(patient):
                for w in frame.winfo_children():
                    w.destroy()
                info = (f"Name: {patient.get('name','')} {patient.get('surname','')}\n"
                        f"DOB: {patient.get('dob','')}\n"
                        f"Gender: {patient.get('gender','N/A')}\n"
                        f"Phone: {patient.get('contact','N/A')}\n"
                        f"Email: {patient.get('email','N/A')}")
                tk.Label(frame, text="Patient Information", font=("Arial",12,"bold")).pack()
                tk.Label(frame, text=info, justify="left").pack(pady=5)
                tk.Label(frame, text="Session History", font=("Arial",12,"bold")).pack(pady=10)
                pnotes = [n for n in notes if n.get('patient_id') == patient.get('id')]
                pnotes.sort(key=lambda n: n.get('session_date',''), reverse=True)
                if not pnotes:
                    tk.Label(frame, text="No session history available.", fg="red").pack()
                    return
                txt = tk.Text(frame, width=70, height=15)
                txt.pack(fill='both', expand=True)
                for n in pnotes:
                    seg = (f"Date: {n.get('session_date','')}\n"
                            f"Subjective: {n.get('subjective','')}\n"
                            f"Objective: {n.get('objective','')}\n"
                            f"Assessment: {n.get('assessment','')}\n"
                            f"Plan: {n.get('plan','')}\n" + "-"*40 + "\n")
                    txt.insert(tk.END, seg)
                txt.config(state="disabled")
            if len(matches) > 1:
                lb = tk.Listbox(frame, height=5)
                for p in matches:
                    lb.insert(tk.END, f"{p.get('name','')} {p.get('surname','')} ({p.get('dob','')})")
                lb.pack(pady=5)
                tk.Button(frame, text="Select", command=lambda: show_history(matches[lb.curselection()[0]])).pack()
            else:
                show_history(matches[0])

        tk.Button(history_win, text="Search", command=search).pack(pady=5)

    def create_patient_frame(self):
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient='vertical', command=canvas.yview)

        canvas.create_window((0, 0), window=scrollable_frame, anchor='nw')
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        self.frames['patient'] = scrollable_frame
        self.bind_scroll_events(canvas)

        tk.Label(scrollable_frame, text='Select Patient to Edit:').grid(row=0, column=0, padx=5, pady=5, sticky='w')
        self.edit_patient_menu = tk.OptionMenu(scrollable_frame, self.edit_patient_var, *self.get_patient_names(), command=self.load_patient_details)
        self.edit_patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky='ew')

        fields = [
            ('Name:', tk.Entry), ('Contact:', tk.Entry), ('DOB (YYYY-MM-DD):', tk.Entry),
            ('Gender:', tk.Entry), ('Medical Aid:', tk.Entry), ('Medical Aid Number:', tk.Entry),
            ('Occupation:', tk.Entry), ('Referral Diagnosis:', tk.Entry), ('Physiotherapist Name:', tk.Entry)
        ]
        self.patient_entries = {}
        for i, (label, widget_type) in enumerate(fields):
            tk.Label(scrollable_frame, text=label).grid(row=i+1, column=0, padx=5, pady=5, sticky='w')
            entry = widget_type(scrollable_frame)
            entry.grid(row=i+1, column=1, padx=5, pady=5, sticky='ew')
            self.patient_entries[label.strip(':')] = entry

        tk.Label(scrollable_frame, text='Medical History').grid(row=len(fields)+1, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text='Conditions (comma-separated):').grid(row=len(fields)+2, column=0, padx=5, pady=5, sticky='w')
        self.conditions_entry = tk.Entry(scrollable_frame)
        self.conditions_entry.grid(row=len(fields)+2, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Surgeries (comma-separated):').grid(row=len(fields)+3, column=0, padx=5, pady=5, sticky='w')
        self.surgeries_entry = tk.Entry(scrollable_frame)
        self.surgeries_entry.grid(row=len(fields)+3, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Allergies (comma-separated):').grid(row=len(fields)+4, column=0, padx=5, pady=5, sticky='w')
        self.allergies_entry = tk.Entry(scrollable_frame)
        self.allergies_entry.grid(row=len(fields)+4, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Diagnosis').grid(row=len(fields)+5, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text='Diagnosis:').grid(row=len(fields)+6, column=0, padx=5, pady=5, sticky='w')
        self.diagnosis_entry = tk.Entry(scrollable_frame)
        self.diagnosis_entry.grid(row=len(fields)+6, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='ICD Code:').grid(row=len(fields)+7, column=0, padx=5, pady=5, sticky='w')
        self.icd_code_entry = tk.Entry(scrollable_frame)
        self.icd_code_entry.grid(row=len(fields)+7, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Treatment Plan').grid(row=len(fields)+8, column=0, columnspan=2, pady=10)
        tk.Label(scrollable_frame, text='Exercises (comma-separated):').grid(row=len(fields)+9, column=0, padx=5, pady=5, sticky='w')
        self.exercises_entry = tk.Entry(scrollable_frame)
        self.exercises_entry.grid(row=len(fields)+9, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Therapies (comma-separated):').grid(row=len(fields)+10, column=0, padx=5, pady=5, sticky='w')
        self.therapies_entry = tk.Entry(scrollable_frame)
        self.therapies_entry.grid(row=len(fields)+10, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Modalities (comma-separated):').grid(row=len(fields)+11, column=0, padx=5, pady=5, sticky='w')
        self.modalities_entry = tk.Entry(scrollable_frame)
        self.modalities_entry.grid(row=len(fields)+11, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Interventions:').grid(row=len(fields)+12, column=0, padx=5, pady=5, sticky='w')
        self.interventions_entry = tk.Entry(scrollable_frame)
        self.interventions_entry.grid(row=len(fields)+12, column=1, padx=5, pady=5, sticky='ew')
        tk.Label(scrollable_frame, text='Home Exercise Program:').grid(row=len(fields)+13, column=0, padx=5, pady=5, sticky='w')
        self.home_exercise_entry = tk.Entry(scrollable_frame)
        self.home_exercise_entry.grid(row=len(fields)+13, column=1, padx=5, pady=5, sticky='ew')

        tk.Button(scrollable_frame, text='Add Patient', command=self.add_patient).grid(row=len(fields)+14, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Edit Patient', command=self.edit_patient).grid(row=len(fields)+15, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Back to Menu', command=self.create_main_menu, width=20).grid(row=len(fields)+16, column=0, columnspan=2, pady=20)

        scrollable_frame.columnconfigure(1, weight=1)

    def create_assessment_frame(self):
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient='vertical', command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        scrollable_frame.bind(
            '<Configure>',
            lambda e: canvas.configure(scrollregion=canvas.bbox('all'))
        )
        canvas.create_window((0, 0), window=scrollable_frame, anchor='nw')
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        self.frames['assessment'] = scrollable_frame
        self.bind_scroll_events(canvas)

        tk.Label(scrollable_frame, text='Select Patient:').grid(row=0, column=0, padx=5, pady=5, sticky='w')
        self.assess_patient_menu = tk.OptionMenu(scrollable_frame, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.assess_patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Assessment Findings:').grid(row=1, column=0, padx=5, pady=5, sticky='w')
        self.assessment_findings_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.assessment_findings_entry.grid(row=1, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Subjective Symptoms:').grid(row=2, column=0, padx=5, pady=5, sticky='w')
        self.subjective_symptoms_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.subjective_symptoms_entry.grid(row=2, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Objective Findings:').grid(row=3, column=0, padx=5, pady=5, sticky='w')
        self.objective_findings_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.objective_findings_entry.grid(row=3, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='History:').grid(row=4, column=0, padx=5, pady=5, sticky='w')
        self.history_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.history_entry.grid(row=4, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Treatment Notes (Session):').grid(row=5, column=0, padx=5, pady=5, sticky='w')
        self.treatment_notes_entry = tk.Text(scrollable_frame, height=5, width=40)
        self.treatment_notes_entry.grid(row=5, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Reassessment Progress Notes:').grid(row=6, column=0, padx=5, pady=5, sticky='w')
        self.reassessment_progress_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.reassessment_progress_entry.grid(row=6, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Reassessment Objective Findings:').grid(row=7, column=0, padx=5, pady=5, sticky='w')
        self.reassessment_objective_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.reassessment_objective_entry.grid(row=7, column=1, padx=5, pady=5, sticky='ew')

        tk.Label(scrollable_frame, text='Recommendations:').grid(row=8, column=0, padx=5, pady=5, sticky='w')
        self.recommendations_entry = tk.Text(scrollable_frame, height=3, width=40)
        self.recommendations_entry.grid(row=8, column=1, padx=5, pady=5, sticky='ew')

        tk.Button(scrollable_frame, text='Add Assessment', command=self.add_assessment).grid(row=9, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Add Reassessment', command=self.add_reassessment).grid(row=10, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Back to Menu', command=self.create_main_menu, width=20).grid(row=11, column=0, columnspan=2, pady=20)

        scrollable_frame.columnconfigure(1, weight=1)

    def create_report_frame(self):
        self.clear_container()
        canvas = tk.Canvas(self.container)
        scrollbar = ttk.Scrollbar(self.container, orient='vertical', command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        scrollable_frame.bind(
            '<Configure>',
            lambda e: canvas.configure(scrollregion=canvas.bbox('all'))
        )
        canvas.create_window((0, 0), window=scrollable_frame, anchor='nw')
        canvas.configure(yscrollcommand=scrollbar.set)
        canvas.pack(side='left', fill='both', expand=True)
        scrollbar.pack(side='right', fill='y')
        self.frames['report'] = scrollable_frame
        self.bind_scroll_events(canvas)

        tk.Label(scrollable_frame, text='Select Patient for Report:').grid(row=0, column=0, padx=5, pady=5, sticky='w')
        self.report_patient_menu = tk.OptionMenu(scrollable_frame, self.patient_var, *self.get_patient_names(), command=self.select_patient)
        self.report_patient_menu.grid(row=0, column=1, padx=5, pady=5, sticky='ew')

        tk.Button(scrollable_frame, text='Generate Word Report', command=self.generate_word_report).grid(row=1, column=0, columnspan=2, pady=10)
        tk.Button(scrollable_frame, text='Back to Menu', command=self.create_main_menu, width=20).grid(row=2, column=0, columnspan=2, pady=20)

        scrollable_frame.columnconfigure(1, weight=1)

    def add_patient(self):
        try:
            name = self.patient_entries.get('Name', tk.Entry(self.frames['patient'])).get()
            contact = self.patient_entries.get('Contact', tk.Entry(self.frames['patient'])).get()
            dob_str = self.patient_entries.get('DOB (YYYY-MM-DD)', tk.Entry(self.frames['patient'])).get()
            gender = self.patient_entries.get('Gender', tk.Entry(self.frames['patient'])).get()
            medical_aid = self.patient_entries.get('Medical Aid', tk.Entry(self.frames['patient'])).get()
            medical_aid_number = self.patient_entries.get('Medical Aid Number', tk.Entry(self.frames['patient'])).get()
            occupation = self.patient_entries.get('Occupation', tk.Entry(self.frames['patient'])).get()
            referral_diagnosis = self.patient_entries.get('Referral Diagnosis', tk.Entry(self.frames['patient'])).get()
            physiotherapist_name = self.patient_entries.get('Physiotherapist Name', tk.Entry(self.frames['patient'])).get()

            try:
                dob = datetime.strptime(dob_str, '%Y-%m-%d').date()
            except ValueError:
                messagebox.showerror('Error', 'Invalid DOB format! Use YYYY-MM-DD.')
                return

            conditions = [c.strip() for c in self.conditions_entry.get().split(',') if c.strip()] or ['none']
            surgeries = [s.strip() for s in self.surgeries_entry.get().split(',') if s.strip()] or ['none']
            allergies = [a.strip() for a in self.allergies_entry.get().split(',') if a.strip()] or ['none']
            diagnosis = self.diagnosis_entry.get() or 'N/A'
            icd_code = self.icd_code_entry.get() or 'N/A'
            exercises = [e.strip() for e in self.exercises_entry.get().split(',') if e.strip()] or ['none']
            therapies = [t.strip() for t in self.therapies_entry.get().split(',') if t.strip()] or ['none']
            modalities = [m.strip() for m in self.modalities_entry.get().split(',') if m.strip()] or ['none']
            interventions = self.interventions_entry.get() or 'Initial treatment'
            home_exercise = self.home_exercise_entry.get() or 'Home exercises prescribed'

            if not name or not referral_diagnosis or not physiotherapist_name:
                messagebox.showerror('Error', 'Name, referral diagnosis, and physiotherapist name are required!')
                return

            medical_history = MedicalHistory(conditions, surgeries, allergies)
            diagnosis_obj = Diagnosis(diagnosis, icd_code)
            treatment_plan = TreatmentPlan(exercises, therapies, modalities, interventions, home_exercise)

            patient = Patient(
                name, contact, dob, gender, medical_aid, medical_aid_number, occupation,
                referral_diagnosis, physiotherapist_name, medical_history, diagnosis_obj, treatment_plan
            )

            patient_id = f'patient_{self.patient_counter:03d}'
            self.patients[patient_id] = patient
            self.patient_counter += 1
            self.current_patient = patient
            self.save_patients()
            self.update_patient_menu()
            messagebox.showinfo('Success', f'Patient {name} added!')
            self.clear_patient_entries()

        except Exception as e:
            logging.error(f'Failed to add patient: {str(e)}')
            messagebox.showerror('Error', f'Failed to add patient: {str(e)}')

    def edit_patient(self):
        patient_name = self.edit_patient_var.get()
        patient = next((p for p in self.patients.values() if p.name == patient_name), None)
        if not patient:
            messagebox.showerror('Error', 'No patient selected!')
            return

        try:
            name = self.patient_entries.get('Name', tk.Entry(self.frames['patient'])).get()
            contact = self.patient_entries.get('Contact', tk.Entry(self.frames['patient'])).get()
            dob_str = self.patient_entries.get('DOB (YYYY-MM-DD)', tk.Entry(self.frames['patient'])).get()
            gender = self.patient_entries.get('Gender', tk.Entry(self.frames['patient'])).get()
            medical_aid = self.patient_entries.get('Medical Aid', tk.Entry(self.frames['patient'])).get()
            medical_aid_number = self.patient_entries.get('Medical Aid Number', tk.Entry(self.frames['patient'])).get()
            occupation = self.patient_entries.get('Occupation', tk.Entry(self.frames['patient'])).get()
            referral_diagnosis = self.patient_entries.get('Referral Diagnosis', tk.Entry(self.frames['patient'])).get()
            physiotherapist_name = self.patient_entries.get('Physiotherapist Name', tk.Entry(self.frames['patient'])).get()

            try:
                dob = datetime.strptime(dob_str, '%Y-%m-%d').date()
            except ValueError:
                messagebox.showerror('Error', 'Invalid DOB format! Use YYYY-MM-DD.')
                return

            conditions = [c.strip() for c in self.conditions_entry.get().split(',') if c.strip()] or ['none']
            surgeries = [s.strip() for s in self.surgeries_entry.get().split(',') if s.strip()] or ['none']
            allergies = [a.strip() for a in self.allergies_entry.get().split(',') if a.strip()] or ['none']
            diagnosis = self.diagnosis_entry.get() or 'N/A'
            icd_code = self.icd_code_entry.get() or 'N/A'
            exercises = [e.strip() for e in self.exercises_entry.get().split(',') if e.strip()] or ['none']
            therapies = [t.strip() for t in self.therapies_entry.get().split(',') if t.strip()] or ['none']
            modalities = [m.strip() for m in self.modalities_entry.get().split(',') if m.strip()] or ['none']
            interventions = self.interventions_entry.get() or 'Initial treatment'
            home_exercise = self.home_exercise_entry.get() or 'Home exercises prescribed'

            if not name or not referral_diagnosis or not physiotherapist_name:
                messagebox.showerror('Error', 'Name, referral diagnosis, and physiotherapist name are required!')
                return

            patient.name = name
            patient.contact = contact
            patient.dob = dob
            patient.gender = gender
            patient.medical_aid = medical_aid
            patient.medical_aid_number = medical_aid_number
            patient.occupation = occupation
            patient.referral_diagnosis = referral_diagnosis
            patient.physiotherapist_name = physiotherapist_name
            patient.medical_history = MedicalHistory(conditions, surgeries, allergies)
            patient.diagnosis = Diagnosis(diagnosis, icd_code)
            patient.treatment_plan = TreatmentPlan(exercises, therapies, modalities, interventions, home_exercise)

            self.save_patients()
            self.update_patient_menu()
            messagebox.showinfo('Success', f'Patient {name} updated!')
            self.clear_patient_entries()

        except Exception as e:
            logging.error(f'Failed to update patient: {str(e)}')
            messagebox.showerror('Error', f'Failed to update patient: {str(e)}')

    def load_patient_details(self, *args):
        patient_name = self.edit_patient_var.get()
        patient = next((p for p in self.patients.values() if p.name == patient_name), None)
        if not patient:
            self.clear_patient_entries()
            return

        self.patient_entries['Name'].delete(0, tk.END)
        self.patient_entries['Name'].insert(0, patient.name)
        self.patient_entries['Contact'].delete(0, tk.END)
        self.patient_entries['Contact'].insert(0, patient.contact)
        self.patient_entries['DOB (YYYY-MM-DD)'].delete(0, tk.END)
        self.patient_entries['DOB (YYYY-MM-DD)'].insert(0, str(patient.dob))
        self.patient_entries['Gender'].delete(0, tk.END)
        self.patient_entries['Gender'].insert(0, patient.gender)
        self.patient_entries['Medical Aid'].delete(0, tk.END)
        self.patient_entries['Medical Aid'].insert(0, patient.medical_aid)
        self.patient_entries['Medical Aid Number'].delete(0, tk.END)
        self.patient_entries['Medical Aid Number'].insert(0, patient.medical_aid_number)
        self.patient_entries['Occupation'].delete(0, tk.END)
        self.patient_entries['Occupation'].insert(0, patient.occupation)
        self.patient_entries['Referral Diagnosis'].delete(0, tk.END)
        self.patient_entries['Referral Diagnosis'].insert(0, patient.referral_diagnosis)
        self.patient_entries['Physiotherapist Name'].delete(0, tk.END)
        self.patient_entries['Physiotherapist Name'].insert(0, patient.physiotherapist_name)

        self.conditions_entry.delete(0, tk.END)
        self.conditions_entry.insert(0, ', '.join(patient.medical_history.conditions))
        self.surgeries_entry.delete(0, tk.END)
        self.surgeries_entry.insert(0, ', '.join(patient.medical_history.surgeries))
        self.allergies_entry.delete(0, tk.END)
        self.allergies_entry.insert(0, ', '.join(patient.medical_history.allergies))

        self.diagnosis_entry.delete(0, tk.END)
        self.diagnosis_entry.insert(0, patient.diagnosis.diagnosis)
        self.icd_code_entry.delete(0, tk.END)
        self.icd_code_entry.insert(0, patient.diagnosis.icd_code)

        self.exercises_entry.delete(0, tk.END)
        self.exercises_entry.insert(0, ', '.join(patient.treatment_plan.exercises))
        self.therapies_entry.delete(0, tk.END)
        self.therapies_entry.insert(0, ', '.join(patient.treatment_plan.therapies))
        self.modalities_entry.delete(0, tk.END)
        self.modalities_entry.insert(0, ', '.join(patient.treatment_plan.modalities))
        self.interventions_entry.delete(0, tk.END)
        self.interventions_entry.insert(0, patient.treatment_plan.interventions)
        self.home_exercise_entry.delete(0, tk.END)
        self.home_exercise_entry.insert(0, patient.treatment_plan.home_exercise_program)

    def clear_patient_entries(self):
        for entry in self.patient_entries.values():
            entry.delete(0, tk.END)
        self.conditions_entry.delete(0, tk.END)
        self.surgeries_entry.delete(0, tk.END)
        self.allergies_entry.delete(0, tk.END)
        self.diagnosis_entry.delete(0, tk.END)
        self.icd_code_entry.delete(0, tk.END)
        self.exercises_entry.delete(0, tk.END)
        self.therapies_entry.delete(0, tk.END)
        self.modalities_entry.delete(0, tk.END)
        self.interventions_entry.delete(0, tk.END)
        self.home_exercise_entry.delete(0, tk.END)

    def add_assessment(self):
        if not self.current_patient:
            messagebox.showerror('Error', 'No patient selected!')
            return

        findings = self.assessment_findings_entry.get('1.0', tk.END).strip()
        subjective_symptoms = self.subjective_symptoms_entry.get('1.0', tk.END).strip()
        objective_findings = self.objective_findings_entry.get('1.0', tk.END).strip()
        history = self.history_entry.get('1.0', tk.END).strip()
        treatment_notes = self.treatment_notes_entry.get('1.0', tk.END).strip()

        if not all([findings, subjective_symptoms, objective_findings]):
            messagebox.showerror('Error', 'Findings, Subjective Symptoms, and Objective Findings are required!')
            return

        assessment = Assessment(findings, date.today(), subjective_symptoms, objective_findings, history or 'N/A', treatment_notes or 'N/A')
        self.current_patient.add_assessment(assessment)
        self.save_patients()
        messagebox.showinfo('Success', 'Assessment added!')

        self.assessment_findings_entry.delete('1.0', tk.END)
        self.subjective_symptoms_entry.delete('1.0', tk.END)
        self.objective_findings_entry.delete('1.0', tk.END)
        self.history_entry.delete('1.0', tk.END)
        self.treatment_notes_entry.delete('1.0', tk.END)

    def add_reassessment(self):
        if not self.current_patient:
            messagebox.showerror('Error', 'No patient selected!')
            return

        progress_notes = self.reassessment_progress_entry.get('1.0', tk.END).strip()
        objective_findings = self.reassessment_objective_entry.get('1.0', tk.END).strip()
        recommendations = self.recommendations_entry.get('1.0', tk.END).strip()

        if not all([progress_notes, objective_findings]):
            messagebox.showerror('Error', 'Progress Notes and Objective Findings are required!')
            return

        reassessment = Reassessment(date.today(), progress_notes, objective_findings, recommendations or 'N/A')
        self.current_patient.add_reassessment(reassessment)
        self.save_patients()
        messagebox.showinfo('Success', 'Reassessment added!')

        self.reassessment_progress_entry.delete('1.0', tk.END)
        self.reassessment_objective_entry.delete('1.0', tk.END)
        self.recommendations_entry.delete('1.0', tk.END)

    def generate_word_report(self):
        if not self.current_patient:
            messagebox.showerror('Error', 'No patient selected!')
            return

        document = Document()
        document.add_heading(f'Physiotherapy Management Report for {self.current_patient.name}', level=0)

        document.add_paragraph(f'Name: {self.current_patient.name}')
        document.add_paragraph(f'Date of Birth: {self.current_patient.dob}')
        document.add_paragraph(f'Medical Aid: {self.current_patient.medical_aid}')
        document.add_paragraph(f'Medical Aid Number: {self.current_patient.medical_aid_number}')
        document.add_paragraph(f'Occupation: {self.current_patient.occupation}')
        document.add_paragraph(f'Physiotherapist: {self.current_patient.physiotherapist_name}')
        document.add_paragraph(f'Referral Diagnosis: {self.current_patient.referral_diagnosis}')

        document.add_paragraph('Dear Dr,')
        document.add_paragraph(f'Thank you for the referral of the above-mentioned patient.')

        age = (datetime.now().date() - self.current_patient.dob).days // 365
        age_str = f'{age}-year-old' if age >= 1 else f'{(datetime.now().date() - self.current_patient.dob).days // 30}-month-old'

        narrative = (
            f'The patient is a {age_str} {self.current_patient.gender.lower()} who presented '
            f'with {self.current_patient.referral_diagnosis.lower()}. '
        )

        if self.current_patient.assessments:
            latest_assessment = self.current_patient.assessments[-1]
            narrative += (
                f'On assessment, the patient reported {latest_assessment.subjective_symptoms.lower()}. '
                f'Objective assessment revealed {latest_assessment.objective_findings.lower()}. '
                f'Relevant history includes {latest_assessment.history.lower()}. '
            )
        else:
            narrative += 'No assessment has been recorded. '

        narrative += (
            f'Physiotherapy management included {self.current_patient.treatment_plan.interventions.lower()}. '
        )
        if self.current_patient.assessments and self.current_patient.assessments[-1].treatment_notes != 'N/A':
            narrative += (
                f'During the session, the following treatments were applied: '
                f'{self.current_patient.assessments[-1].treatment_notes.lower()}. '
            )
        narrative += (
            f'A home exercise program was provided, including '
            f'{self.current_patient.treatment_plan.home_exercise_program.lower()}. '
        )

        if self.current_patient.reassessments:
            latest_reassessment = self.current_patient.reassessments[-1]
            narrative += (
                f'On reassessment, the patient reported {latest_reassessment.progress_notes.lower()}. '
                f'Objective findings included {latest_reassessment.objective_findings.lower()}. '
                f'{latest_reassessment.recommendations}.'
            )
        else:
            narrative += 'No reassessment has been recorded.'

        document.add_paragraph(narrative.strip())

        document.add_paragraph('Kind regards,')
        document.add_paragraph(self.current_patient.physiotherapist_name)
        document.add_paragraph('Physiotherapist')

        try:
            filename = f'{self.current_patient.name.replace(" ", "_")}_report.docx'
            document.save(filename)
            messagebox.showinfo('Success', f'Report saved as "{filename}"')
        except Exception as e:
            logging.error(f'Failed to save report: {str(e)}')
            messagebox.showerror('Error', f'Failed to save report: {str(e)}')

    def get_patient_names(self):
        return [patient.name for patient in self.patients.values()] or ['No patients']

    def select_patient(self, *args):
        patient_name = self.patient_var.get()
        self.current_patient = next((p for p in self.patients.values() if p.name == patient_name), None)

    def save_patients(self):
        try:
            patients_data = {pid: patient.to_dict() for pid, patient in self.patients.items()}
            with open('patients.json', 'w', encoding='utf-8') as f:
                json.dump(patients_data, f, default=str)
        except Exception as e:
            logging.error(f'Failed to save patients: {str(e)}')
            messagebox.showerror('Error', f'Failed to save patients: {str(e)}')

    def load_patients(self):
        try:
            with open('patients.json', 'r', encoding='utf-8') as f:
                patients_data = json.load(f, object_hook=str_to_date)
            self.patients = {}
            for pid, data in patients_data.items():
                try:
                    self.patients[pid] = Patient.from_dict(data)
                except Exception as e:
                    logging.error(f'Error loading patient {pid}: {str(e)}')
                    messagebox.showwarning('Warning', f'Failed to load patient {pid}: {str(e)}')
            self.patient_counter = len(self.patients) + 1
            if self.patients:
                self.current_patient = list(self.patients.values())[-1]
            self.update_patient_menu()
        except FileNotFoundError:
            self.patients = {}
            self.update_patient_menu()
        except Exception as e:
            logging.error(f'Failed to load patients: {str(e)}')
            messagebox.showerror('Error', f'Failed to load patients: {str(e)}')

    def update_patient_menu(self):
        for frame_name, menu_var in [('assessment', 'assess_patient_menu'), ('report', 'report_patient_menu'), ('patient', 'edit_patient_menu')]:
            if frame_name in self.frames and hasattr(self, menu_var):
                menu = getattr(self, menu_var)['menu']
                menu.delete(0, 'end')
                for name in self.get_patient_names():
                    menu.add_command(label=name, command=lambda n=name: getattr(self, menu_var.replace('menu', 'var')).set(n))
        if self.get_patient_names() != ['No patients']:
            self.patient_var.set(self.get_patient_names()[-1])
            if hasattr(self, 'edit_patient_var'):
                self.edit_patient_var.set(self.get_patient_names()[-1])
        else:
            self.patient_var.set('No patients')
            if hasattr(self, 'edit_patient_var'):
                self.edit_patient_var.set('No patients')

    def load_patients_data(self):
        """Return list of patients from JSON, or [] on error."""
        try:
            with open('patients.json', 'r', encoding='utf-8') as f:
                return json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            messagebox.showerror("Error", "Unable to load patients data.")
            return []

    def load_notes_data(self):
        """Return list of session notes from JSON, or [] on error."""
        try:
            with open('notes.json', 'r', encoding='utf-8') as f:
                return json.load(f)
        except (FileNotFoundError, json.JSONDecodeError):
            messagebox.showerror("Error", "Unable to load notes data.")
            return []

    def view_patient_history(self):
        """Popup window showing patient info + past session notes."""
        patients = self.load_patients_data()
        notes = self.load_notes_data()

        history_win = tk.Toplevel(self.root)
        history_win.title("View Patient History")
        history_win.geometry("600x600")

        # --- Search bar ---
        tk.Label(history_win, text="Enter Patient Name:").pack(pady=5)
        entry = tk.Entry(history_win, width=40)
        entry.pack(pady=5)

        # Frame for results
        frame = tk.Frame(history_win)
        frame.pack(fill='both', expand=True, pady=10)

        def search():
            # Clear previous
            for w in frame.winfo_children():
                w.destroy()

            query = entry.get().lower().strip()
            matches = [p for p in patients
                       if query in (p['name'] + " " + p.get('surname','')).lower()]

            if not matches:
                tk.Label(frame, text="No patients found.", fg="red").pack()
                return

            def show_history(patient):
                # clear frame
                for w in frame.winfo_children():
                    w.destroy()

                info = (
                    f"Name: {patient['name']} {patient.get('surname','')}\n"
                    f"DOB: {patient['dob']}\n"
                    f"Gender: {patient.get('gender','N/A')}\n"
                    f"Phone: {patient.get('phone','N/A')}\n"
                    f"Email: {patient.get('email','N/A')}"
                )
                tk.Label(frame, text="Patient Information", font=("Arial",12,"bold")).pack()
                tk.Label(frame, text=info, justify="left").pack(pady=5)

                tk.Label(frame, text="Session History", font=("Arial",12,"bold")).pack(pady=10)

                pnotes = [n for n in notes if n['patient_id']==patient['id']]
                pnotes.sort(key=lambda n: n['session_date'], reverse=True)

                if not pnotes:
                    tk.Label(frame, text="No session history available.", fg="red").pack()
                    return

                txt = tk.Text(frame, width=70, height=15)
                txt.pack(fill='both', expand=True)
                for n in pnotes:
                    seg = (
                        f"Date: {n['session_date']}\n"
                        f"Subj: {n['subjective']}\n"
                        f"Obj: {n['objective']}\n"
                        f"Assess: {n['assessment']}\n"
                        f"Plan: {n['plan']}\n"
                        + "-"*40 + "\n"
                    )
                    txt.insert(tk.END, seg)
                txt.config(state="disabled")

            # if multiple matches
            if len(matches) > 1:
                lb = tk.Listbox(frame, height=5)
                for p in matches:
                    lb.insert(tk.END, f"{p['name']} {p.get('surname','')} ({p['dob']})")
                lb.pack(pady=5)
                tk.Button(frame, text="Select", command=lambda: show_history(matches[lb.curselection()[0]])).pack()
            else:
                show_history(matches[0])

        tk.Button(history_win, text="Search", command=search).pack(pady=5)


    # Function to search and display results
    def search_patient():
        for widget in history_frame.winfo_children():
            widget.destroy()

        name_query = patient_name_entry.get().lower().strip()
        matched_patients = [p for p in patients if name_query in (p['name'] + " " + p.get('surname', '')).lower()]

        if not matched_patients:
            tk.Label(history_frame, text="No patients found.", fg="red").pack()
            return

        # If multiple matches, show a selection list
        if len(matched_patients) > 1:
            tk.Label(history_frame, text="Multiple matches found. Select one:", fg="blue").pack()
            selection_box = tk.Listbox(history_frame, height=5)
            for patient in matched_patients:
                selection_box.insert(tk.END, f"{patient['name']} {patient.get('surname', '')} ({patient['dob']})")
            selection_box.pack(pady=5)

            def select_patient():
                index = selection_box.curselection()
                if not index:
                    messagebox.showerror("Error", "Please select a patient.")
                    return
                display_patient_history(matched_patients[index[0]])

            tk.Button(history_frame, text="Select", command=select_patient).pack(pady=5)
        else:
            display_patient_history(matched_patients[0])

    # Display patient info and notes
    def display_patient_history(patient):
        for widget in history_frame.winfo_children():
            widget.destroy()

        patient_info = (
            f"Name: {patient['name']} {patient.get('surname', '')}\n"
            f"DOB: {patient['dob']}\n"
            f"Gender: {patient.get('gender', 'N/A')}\n"
            f"Phone: {patient.get('phone', 'N/A')}\n"
            f"Email: {patient.get('email', 'N/A')}"
        )
        tk.Label(history_frame, text="Patient Information", font=("Arial", 12, "bold")).pack(pady=5)
        tk.Label(history_frame, text=patient_info, justify="left").pack(pady=5)

        tk.Label(history_frame, text="Session History", font=("Arial", 12, "bold")).pack(pady=10)

        patient_notes = [n for n in notes if n['patient_id'] == patient['id']]
        patient_notes.sort(key=lambda x: x['session_date'], reverse=True)

        if not patient_notes:
            tk.Label(history_frame, text="No session history available.", fg="red").pack()
            return

        # Add a scrollable text widget to display notes
        notes_text = tk.Text(history_frame, width=70, height=15)
        notes_text.pack(pady=5, fill="both", expand=True)

        for note in patient_notes:
            session_info = (
                f"Session Date: {note['session_date']}\n"
                f"Subjective: {note['subjective']}\n"
                f"Objective: {note['objective']}\n"
                f"Assessment: {note['assessment']}\n"
                f"Plan: {note['plan']}\n"
                "—" * 40 + "\n"
            )
            notes_text.insert(tk.END, session_info)
        notes_text.config(state="disabled")

    # Search button
    tk.Button(history_win, text="Search", command=search_patient).pack(pady=5)
    # In your main Tkinter window setup:
    view_history_button = tk.Button(main_window, text="View Patient History", command=view_patient_history)
    view_history_button.pack(pady=10)

    # Frame to hold patient details and notes
    history_frame = tk.Frame(history_win)
    history_frame.pack(pady=10, fill="both", expand=True)


if __name__ == '__main__':
    root = tk.Tk()
    app = PhysioApp(root)
    root.mainloop()

NameError: name 'history_win' is not defined